# Can we use bioinformatics to tell how closely related pathogens are?

In bioinformatics, we (the scientists) often want to compare many different bacteria, viruses, and fungi to understand their related functions. Think about bioinformatics as analyzing the blueprints for all organisms, allowing us to understand more throughly how things work down to the most basic building blocks of life.

Bioinformaticians use many tools to understand the organisms they work with. Today, we will use k-mer distance estimation, a powerful analysis technique that can be used to estimate how similar multiple organism are to one another. By breaking down the genetic sequences of these organisms into smaller fragments known as k-mers, we can find patterns across the genome that make organisms act uniquely, affecting how an organism can act as a pathogen, as well as how it adapts to its environment.

In this module, we will compare several different bacteria and viruses to get an understanding of how closely related these pathogens are. Background on some common human pathogens is found below:

### E. coli (Escherichia coli):
E. coli is a bacteria commonly found in the human GI tract. Some types of E. coli have a gene for a toxin that can make us sick, especially if found in contaminated food or water.

### S. aureus (Staphylococcus aureus):
S. aureus is a bacteria commonly found on human skin. Some types of S. aureus, like MRSA, have become resistant to many antibiotics, and can cause serious skin infections.

### S. pyogenes (Streptococcus pyogenes):
S. pyogenes, also known as Group A strep, is the main cause of strep throat, and can typically be treated with penicllin.

### SARS-CoV-2 (Severe Acute Respiratory Syndrome Coronavirus 2):
SARS-CoV-2 is the virus that causes COVID-19, and is typically spread through respiratory droplets.

### H3N2 (Influenza H3N2 virus):
H3N2 is the virus that causes influenza type A in humans, but can also affect birds and other mammals. H3N2 is also spread primarily through respiratory droplets. The H3N2 mutates frequently, warranting development of yearly flu shots aimed at the predicted flu subtype.

# ***********************************************************

# Downloading the NCBI RefSeq Databse

NCBI is a centralized hub for genomic data. The NCBI RefSeq database contains genomes of tens of thousands of organisms, carefully curate for public use.

Our first step is to download the RefSeq database.

Click on the cell below and hit the play button to run the cell of code. If the code runs successfully, we should see output saying so.

In [ ]:
import os
import urllib.request
import gzip

os.makedirs("genomes", exist_ok="True")

# Downloads the latest refseq assembly summary if not already downloaded
#if not os.path.exists("assembly_summary_refseq.txt"):
    #urllib.request.urlretrieve("https://ftp.ncbi.nlm.nih.gov/genomes/ASSEMBLY_REPORTS/assembly_summary_refseq.txt", "assembly_summary_refseq.txt")
    
print("NCBI RefSeq database successfully downloaded")

# Retrieving our Pathogen Sequences from RefSeq

We have successfully downloaded the RefSeq database. However, we need to query our database to retrieve our genomes of interest. When the text box pops up, type in Escherichia coli, Staphylococcus aureus, Streptococcus pyogenes, Severe Acute Respiratory Syndrome Coronavirus 2, and Influenza H3N2 virus one by one. Copy and paste if needed since the database will not be able to find a genome in the case of a spelling mistake.

Click on the cell below and hit the play button to run the cell of code.

In [ ]:
def query_database():
    # Retrieves link from RefSeq database
    with gzip.open("assembly_summary_refseq.txt.gz", "rt") as species_database:
        num_genomes = 0
        while True:
            # This handles user input, and stops looping if the user enters nothing
            user_input = input("""Enter a genome you'd like to download and hit enter:
Once you have all the genomes you want, hit enter to continue.\n""")
            if not user_input:
                break
            
            genus, species = user_input.lower().split(" ")[:2]

            # Parse the RefSeq database to find our organism of interst
            link = ""
            for line in species_database:
                if line.startswith("#"):
                    continue
                    
                organism = line.split("\t")[0].lower()
                if genus in organism and species in organism:
                    link = line.split("\t")[1]
                    break
                    
            if not link:
                print(f"No matches found for {genus} {species}")
                continue
            
            num_genomes += 1
            
            # Gets file name
            file_name = link.split("/")[-1] + "_genomic.fna.gz"

            # Gets genome file download link
            fna = f"{link}/{file_name}"

            # Downloads, moves, and unzips file
            urllib.request.urlretrieve(fna, os.path.join("genomes", file_name))
            os.system(f"gunzip {os.path.join('genomes', file_name)}")
            
            print(f"Download complete on {user_input}")
            
    print(f"Successfully downloaded {num_genomes} genomes")
            
query_database()

# The k-mer distance estimation code

The next cell contains the code for the k-mer distance estimation program. This includes code for extracting the k-mers for each of our downloaded genomes, computing distance based on the k-mers, and formatting the output into a heatmap. This cell prepares our program to run, but will not complete any calculations yet.

Click on the cell below and hit the play button to run the cell of code.

In [ ]:
import Bio.SeqIO, math, re, scipy, warnings
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from statistics import pstdev
warnings.filterwarnings("ignore")

# generates kmer dictionary from fasta formatted data
def parse_fasta(fasta, k):
    kmer_dict = {}

    # parses fasta file using SeqRecord iterator
    for record in Bio.SeqIO.parse(fasta, "fasta"):
        seq = record.seq
        regex = "([A-Z][a-z]+)(( [a-z]+)+ ?[1-9]?)"
        match = re.search(regex, record.description)
        try:
            genus = match.group(1)
            species = match.group(2)
            organism = genus + species
        except AttributeError:
            organism = record.description

        # Iterates over all combinations of k continuous nucleotides and adds the kmers to
        # a dictionary with a counter of the number of instances of the kmers
        for nuc_idx in range(len(seq) - (k - 1)):
            kmer = seq[nuc_idx:(nuc_idx + k)]
            kmer_dict.setdefault(kmer, 0)
            kmer_dict[kmer] += 1
            
    print("Parsed {0} -------> Exracted Organism: {1}".format(fasta, organism))
    return (organism, kmer_dict)

# compares the keys in two dictionaries, and adds the missing keys
# from each into one another
def compare_kmer(dict_1, dict_2):
    set_1 = set(dict_1.keys())
    set_2 = set(dict_2.keys())
    diff_keys = list(set_1.symmetric_difference(set_2))
    for key in diff_keys:
        dict_1.setdefault(key, 0)
        dict_2.setdefault(key, 0)

# computes euclidean distance based on two kmer dictionaries
def euclid_kmer(dict_1, dict_2):
    euclid_sum = 0
    for coord in dict_2:
        euclid_sum += (dict_2[coord] - dict_1[coord])**2
    return math.sqrt(euclid_sum)

# normalizes distances in the output matrix to 1
def normalize(distances):
    all_values = []
    for sub in distances:
        for val in sub:
            all_values.append(val)

    minimum = min(all_values)
    maximum = max(all_values)

    # sets values in sub_lists to normalized values
    for sub in distances:
        for idx, val in enumerate(sub):
            norm_val = (val - minimum)/(maximum - minimum)
            sub[idx] = norm_val

    return distances

def matrix_output(samples, k):
    ### {sample_N: {sample_N kmer dict}, ...} ###
    distance_dict = {}
    orgs = []

    # pairs kmer dictionaries with organism names
    for sample in samples:
        org, distance_dict[sample] = parse_fasta(sample, k)
        # adds organism to list for output header
        if org not in orgs:
                orgs.append(org)

    ### {(sample_x, sample_y): dist, ...} ###
    # stores previously computed distance values so program only
    # has to run unique pairs of orgs
    store_compare = {}

    # distance matrix output; list of lists
    distances = []
    for sample_1 in samples:
        sub_dist = []
        for sample_2 in samples:
            # if distance already computed, grab cached value
            if (sample_2, sample_1) in store_compare:
                sub_dist.append(store_compare[sample_2, sample_1])
                continue
            # if samples are the same, dist = 0
            if sample_1 == sample_2:
                sub_dist.append(0.00)
                continue

            # grabs kmer dictionaries from distance_dict
            kmer_1 = distance_dict[sample_1]
            kmer_2 = distance_dict[sample_2]

            compare_kmer(kmer_1, kmer_2)
            # calculates euclidean distance
            dist = euclid_kmer(kmer_1, kmer_2)
            # adds dist to cache dictionary
            store_compare[(sample_1, sample_2)] = dist
            sub_dist.append(dist)

        distances.append(sub_dist)
    return orgs, distances

def make_table(k, orgs, distances):
    # normalizes the data in distances for heatmap output
    D = np.array(normalize(distances))
    
    fig = plt.figure(figsize=(8,8))
    
    # Dendrogram to right of heat map
    ax1 = fig.add_axes([0.91, 0.1, 0.2, 0.6])
    Y = sch.linkage(D, method='single')
    Z1 = sch.dendrogram(Y, orientation='right')
    ax1.set_xticks([]); ax1.set_yticks([])

    # Dendrogram on top of heat map
    ax2 = fig.add_axes([0.3, 0.71, 0.6, 0.2])
    Y = sch.linkage(D, method='single')
    Z2 = sch.dendrogram(Y)
    ax2.set_xticks([]); ax2.set_yticks([])

    # main heat-map
    axmatrix = fig.add_axes([0.3, 0.1, 0.6, 0.6])
    # orients species by distance from one another
    idx1 = Z1['leaves']
    idx2 = Z2['leaves']
    D = D[idx1, :]
    D = D[:, idx2]
    # the actual heat-map
    im = axmatrix.matshow(D, aspect='auto', origin='lower', cmap="RdBu")
    axmatrix.set_xticks([]); axmatrix.set_yticks([])

    # labeling the x-axis
    axmatrix.set_xticks(range(len(idx1)))
    axmatrix.set_xticklabels([orgs[x] for x in idx1], minor=False)
    axmatrix.xaxis.set_label_position('bottom')
    axmatrix.xaxis.tick_bottom()

    plt.xticks(rotation=-90, fontsize=8)

    # labeling the y-axis
    axmatrix.set_yticks(range(len(idx2)))
    axmatrix.set_yticklabels([orgs[x] for x in idx2], minor=False)
    axmatrix.yaxis.set_label_position('left')
    axmatrix.yaxis.tick_left()

    plt.show()
    
print("Successfully loaded k-mer distance estimation code")

# Running the k-mer distance estimation program

The following code cell contains the code necessary to run the program itself. The program should update periodically at it goes through each genome and will output a heatmap at the end.

In [ ]:
genomes = [genome.path for genome in os.scandir("genomes")]
organisms, distances = matrix_output(genomes, 4)
make_table(4, organisms, distances)

# Analyzing the results

[Back to main site](https://lawrencegordon.github.io/)